### This script is created by Hugsney
<p>Automatizacao do processo de pagamento</p>
<br>
<h1>Primeira etapa do script - Lendo Banco de dados</h1>
<h6>celula_1</h6>

In [2]:
# Livrarias importadas
import os
import pandas as pd
import shutil
import xlsxwriter
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle
from reportlab.lib.pagesizes import A4
from reportlab.lib import colors
import time
from datetime import datetime, timedelta
import glob
import zipfile
import xml.etree.ElementTree as ET
from xml.dom import minidom

# Criando uma variavel com a data atualizada
data = datetime.now().strftime('%d-%m-%Y')
data_arquivo = datetime.now().strftime('%Y%m%d')


# Data de ontem
#ontem = datetime.now() - timedelta(days=1)
#data = ontem.strftime('%d-%m-%Y')        # Ex: '30-03-2025'
#data_arquivo = ontem.strftime('%Y%m%d')  # Ex: '20250330'


# Definir o caminho para os arquivos serem encontrados
folder_path = "/Users/hugsneyf/Downloads/outros arquivos/project_Gui_Uber/Historico"
arquivos_semanais = '/Users/hugsneyf/Downloads/outros arquivos/project_Gui_Uber/Arquivos_Bolt_Uber'

# Caminho da nova pasta
new_folder_path = os.path.join(folder_path, f"{data_arquivo}")
if not os.path.exists(new_folder_path):
    os.makedirs(new_folder_path)

def save_df_to_excel(data_frame, file_name, nome_aba='Sheet1'):
    """
    Save a DataFrame to Excel, adjusting the width of the columns.
    
    Parameters:
    - data_frame: The DataFrame to be saved.
    - file_name: The name the file will be saved as.
    - sheet_name: Name of the sheet in the Excel file.
    """

        # Create a folder using the current date
    folder_with_date = os.path.join(new_folder_path)
    os.makedirs(folder_with_date, exist_ok=True)  # Create the folder if it doesn't exist
    
    # Define the full file path
    full_file_path = os.path.join(folder_with_date, file_name)

    # Create a Pandas Excel writer using XlsxWriter as the engine.
    writer = pd.ExcelWriter(full_file_path, engine='xlsxwriter')
    
    # Convert the DataFrame to an XlsxWriter Excel object.
    data_frame.to_excel(writer, sheet_name=nome_aba, index=False)
    
    # Get the xlsxwriter workbook and worksheet objects.
    workbook  = writer.book
    worksheet = writer.sheets[nome_aba]
    
    # Loop through each column and set the width based on the max length in that column.
    # A little extra width is added for padding.
    for i, col in enumerate(data_frame.columns):
        col_str = str(col)
        column_len = data_frame[col].astype(str).str.len().max()
        column_len = max(column_len, len(col_str)) + 2  # Adding a little extra space
        worksheet.set_column(i, i, column_len)
    
    # Close the Pandas Excel writer and output the Excel file.
    writer.close()


#------------------------------------------ Loading the files  ------------------------------------------

###################################  Arquivos de Banco de Dados  #####################################

# Criando variavel do banco de dados
banco_de_dados = pd.read_excel(f'Banco_de_Dados_Guilherme.xlsx')
banco_de_dados['UUID do motorista'] = banco_de_dados['UUID do motorista'].astype(str)

# Replace 'Banco_de_Dados_Guilherme.xlsx' with the path to your Excel file
outras_info = pd.read_excel('Banco_de_Dados_Guilherme.xlsx', 
                            sheet_name='Outras informações')
outras_info['NOME'] = outras_info['NOME'].str.title()


# Extract required columns from Excel
nome_arquivo = outras_info['Novo Nome'].dropna().tolist()
origem_ref = outras_info['Origem_Ref'].dropna().tolist()
uuid_driver = [int(float(x)) for x in outras_info['Uuid_driver'].dropna().tolist()]
first_name = [int(float(x)) for x in outras_info['First_Name'].dropna().tolist()]
last_name = [int(float(x)) for x in outras_info['Last_Name'].dropna().tolist()]
salario_bruto = [int(float(x)) for x in outras_info['Valor_bruto'].dropna().tolist()]

print('\n- Arquivo "banco de dados" e "Outras Informações" carregados com sucesso')


- Arquivo "banco de dados" e "Outras Informações" carregados com sucesso


<h1>Juntar arquivos Uber e Bolt, para identificar novos drives</h1>
<h6>celula_2</h6>

In [3]:
###################################  Arquivo consolidado Uber / Bolt #####################################

# Criar o frame vazio para consolidar os arquivos de pagamento
para_pagamento = []
arquivos_carregados = []
arquivos_faltando = []

# Validar todos os arquivos
for arquivo, origem, first, last, uuid, salario in zip(nome_arquivo, origem_ref, 
                                                       first_name, last_name, 
                                                       uuid_driver, salario_bruto):
    try:
        # Read the CSV file
        arquivos_semanais_temp = os.path.join(arquivos_semanais, f"{arquivo}.csv")
        
        file = pd.read_csv(arquivos_semanais_temp, on_bad_lines='skip')

        # Keep only the specified columns and rename them
        columns_to_keep = [file.columns[uuid], file.columns[first], 
                           file.columns[last], file.columns[salario]]
        file = file[columns_to_keep]
        file.columns = ['UUID do motorista', 'first', 'last', 'Salario Bruto']

        # Add a new column 'nome_arquivo' with the value of 'arquivo'
        file['Arq_origem'] = arquivo
        file['Origem_Ref'] = origem
        
        # Adiciona a list de nome de arquivos
        arquivos_carregados.append(arquivo)
        
        # Append the modified DataFrame to para_pagamento list
        para_pagamento.append(file)
    
    # Verifica arquivo faltando
    except FileNotFoundError:        
        arquivos_faltando.append(arquivo)


# Printa mensagem sobre arquivos faltantes
if len(arquivos_faltando) > 1:
    print(f'\n* Alerta: Não foram encontrados os arquivos: {arquivos_faltando}."*')
elif len(arquivos_faltando) == 1:
    print(f'\n* Alerta: Não foi encontrado o arquivo: {arquivos_faltando}."*')

# Printa mensagem sobre arquivos combinados
if len(arquivos_carregados) > 1:
    print(f'\n - Foram adicionados os arquivos: {arquivos_carregados}.')
elif len(arquivos_carregados) < 1:
    print(f'\n Nao Foram encontrados nenhum arquivo para pagamento')

    
# Concatenate all DataFrames in para_pagamento into a single DataFrame
consolidado_pgto = pd.concat(para_pagamento)
consolidado_pgto['UUID do motorista'] = consolidado_pgto['UUID do motorista'].astype(str)
consolidado_pgto['UUID do motorista'] = consolidado_pgto['UUID do motorista'].str.replace('+', '')
consolidado_pgto = consolidado_pgto[consolidado_pgto['UUID do motorista'] != 'nan']

consolidado_pgto['UUID do motorista'] = consolidado_pgto['UUID do motorista'].str.strip()



# Clean Salario Bruto
consolidado_pgto['Salario Bruto'] = (consolidado_pgto['Salario Bruto'].astype(str).str.replace(',', '.', regex=False))
consolidado_pgto['Salario Bruto'] = pd.to_numeric(consolidado_pgto['Salario Bruto'], errors='coerce')
consolidado_pgto = consolidado_pgto[consolidado_pgto['Salario Bruto'] > 0]

# Group
consolidado_pgto = consolidado_pgto.groupby('UUID do motorista', as_index=False).agg({
    'Salario Bruto': 'sum',
    'first': 'first',
    'last': 'first',
    'Arq_origem': 'first',
    'Origem_Ref': 'first'
})


dupes = consolidado_pgto['UUID do motorista'].duplicated(keep=False).sum()
print(f"Duplicated UUIDs: {dupes}")


# Isola drivers da uber sem registro
driver_sem_registro = consolidado_pgto[~consolidado_pgto['UUID do motorista'].isin(banco_de_dados['UUID do motorista'])]
driver_sem_registro_size = len(driver_sem_registro['UUID do motorista'])

# Checa o tamanho do arquivo, se tiver algo cria um novo arquivo
if driver_sem_registro_size > 0:

    # Excuta a função previaviamente criada para salvar o arquivo
    #save_df_to_excel(driver_sem_registro, f"{data_arquivo}_drivers_sem_registro - DELETE ME.xlsx", 
    #                 'driver_sem_registro_tab')
    
    # Resumir numeros de registros
    num_registros = driver_sem_registro.groupby('Arq_origem').size().reset_index(name='num_registros')
    num_registros = num_registros.to_string(index=False)

    
    # Display the result
    print(f"""\n\n*** Abra o arquivo: 'drivers_sem_registro': {driver_sem_registro_size} registro/s para ser adicionados ao Banco de Dados ***\n\n
            Resumo:
            {num_registros}
          \n\n""")
    
    print(f'\n - Existe um total de {len(consolidado_pgto)} pagamentos a serem feitos.\n\n')
else:
    print(f"\n- Sem novos Drivers para serem registrados.")

    
# Group by the 'Arq_origem' column
consolidado_pgto_resumo = consolidado_pgto.groupby('Arq_origem').size().reset_index(name='Count')

# Print the summary table
#print(consolidado_pgto_resumo)


* Alerta: Não foi encontrado o arquivo: ['PORTALEGRE']."*

 - Foram adicionados os arquivos: ['ACOURES', 'AVEIRO', 'ALGARVE', 'BEJA', 'BOLT DRIVER WINGS', 'BOLT DRIVER WORK', 'BOLT FOOD', 'BRAGA', 'BRAGANCA', 'CASTELO BRANCO', 'COIMBRA', 'EVORA', 'LEIRIA', 'LISBOA BIKE', 'LISBOA MOTO', 'LISBOA RESERVA', 'PEDRO BRAGA', 'PEDRO PORTO', 'PORTO', 'SANTAREM', 'TVDE NICOLY NOVO', 'TVDE VINICIO NOVO', 'VIANA DO CASTELO', 'VILA REAL', 'VISEU'].
Duplicated UUIDs: 0


*** Abra o arquivo: 'drivers_sem_registro': 64 registro/s para ser adicionados ao Banco de Dados ***


            Resumo:
                   Arq_origem  num_registros
          ALGARVE              2
           AVEIRO              1
BOLT DRIVER WINGS              1
 BOLT DRIVER WORK              1
        BOLT FOOD             27
            BRAGA              3
            EVORA              2
      LISBOA BIKE              3
      LISBOA MOTO              3
   LISBOA RESERVA              9
            PORTO              1
 TVDE 

# Planilha geral
<h6>celula_3</h6>

In [4]:
# Organizando os dados e renomeando as colunas da panilha - uber
planilha_geral = pd.merge(consolidado_pgto, banco_de_dados, on="UUID do motorista", how="left").fillna(0)

# Adiciona uma coluna com a data
planilha_geral['Data'] = pd.to_datetime(data, format='%d-%m-%Y')

# Arruma os tipos de dados das colunas numericas
type_float_columns = ['Salario Bruto', 'DESCONTO ALUGUEL SEMANA', 'DESC. TRANSF BANCARIA', 'Deve(+)']

# Remove virgulas e adiciona '.'
planilha_geral[type_float_columns] = planilha_geral[type_float_columns].replace({',': '.'}, regex=True)

# Troca qualquer texto por zero
planilha_geral[type_float_columns] = planilha_geral[type_float_columns].replace({'[^0-9.]': '0'}, regex=True)

# Garante o tipo de dados numerico
planilha_geral[type_float_columns] = planilha_geral[type_float_columns].astype(float)


# ----------------------------------------- Analises ----------------------------------------- #
# Do grupo de pessoas com aluguel foram separadas 2 grupos:
# 1- Todos que possuem desconto de aluguel
com_aluguel = planilha_geral[planilha_geral['DESCONTO ALUGUEL SEMANA'] > 0]

# 1- Cria uma chave para verificar os duplicados
com_aluguel['key'] = com_aluguel['IBAN'].astype(str) + com_aluguel['DESCONTO ALUGUEL SEMANA'].astype(str)

# 1- Mark all duplicates (first and subsequent occurrences) in the 'key' column
com_aluguel['is_duplicate'] = com_aluguel.duplicated('key', keep=False)

# 1.1 - Pessoas com aluguel em duplicidade e que esta marcada com Bolt --> essas foram zerados os descontos
com_aluguel_dup_uber = com_aluguel[(com_aluguel['is_duplicate'] == True) & (com_aluguel['Plataforma'] == 'Bolt')]
com_aluguel_dup_uber['DESCONTO ALUGUEL SEMANA'] = 0


# 1.2.3 - Restante de pessoas com aluguel (SEM DIPLICIDADE E Uber COMO ORIGEM)
com_aluguel_rest = com_aluguel[~((com_aluguel['is_duplicate'] == True) & (com_aluguel['Plataforma'] == 'Bolt'))]
#com_aluguel_dup_rest = com_aluguel[(com_aluguel['is_duplicate'] == True) & (com_aluguel['Plataforma'] != 'Uber')]
#com_aluguel_unique = com_aluguel[(com_aluguel['is_duplicate'] != True)]

# 1- Juntando arquivos de pessoas com alugueis para os devidos calculos
com_aluguel_resolvido = pd.concat([com_aluguel_dup_uber, com_aluguel_rest])

# 1- Limpando arquivo de colunas extras
com_aluguel_resolvido = com_aluguel_resolvido.drop(['is_duplicate', 'key'], axis=1)

# 2- Pessoas sem desconto de aluguel
sem_aluguel = planilha_geral[planilha_geral['DESCONTO ALUGUEL SEMANA'] <= 0]

# Recriando arquivo final: planilha geral
planilha_geral = pd.concat([sem_aluguel, com_aluguel_resolvido])


# Adiciona coluna com a comissao geral, e quando nao ha registrando calcula com 10%
planilha_geral['Comissao_Geral'] = planilha_geral.apply(
    lambda row: (row['Salario Bruto'] * row['Percentual']) if row['Percentual'] > 0 
    else (row['Salario Bruto'] * 0.10), axis=1).round(2)

# Adiciona coluna com o valor final para pagamento
planilha_geral['Valor a Pagar'] = (planilha_geral['Salario Bruto'] - # ele eh o salario bruto - a comissao
                                       planilha_geral['Comissao_Geral'] -    #  - o desconto aluguel semana
                                       planilha_geral['DESC. TRANSF BANCARIA'] -    #  - o desconto imeadiata
                                       planilha_geral['DESCONTO ALUGUEL SEMANA'] + # e soma com algo que vc dever
                                       planilha_geral['Deve(+)']).round(2) 

# Analise de valores negativos
valores_negativos = planilha_geral[planilha_geral['Valor a Pagar'] < 0]

save_df_to_excel(valores_negativos, f"{data_arquivo}_valores_negativos.xlsx", nome_aba='Sheet1')
#save_df_to_excel(planilha_geral, f"{data_arquivo}_planilha_geral.xlsx", nome_aba='Sheet1')

# Filtrando valores positivos
planilha_geral = planilha_geral[planilha_geral['Valor a Pagar'].astype(float) > 0]

# Arrumar nomes inseridos
for col in ['Socio_1', 'Socio_2', 'Socio_3']:
    planilha_geral[col] = planilha_geral[col].str.strip().str.title()

# Adiciona coluna com o valor da comissão a receber
planilha_geral['Valor Comissão'] = (planilha_geral['Comissao_Geral'] +
                                       planilha_geral['DESCONTO ALUGUEL SEMANA']).round(2)

def process_commission_column(df, banco_comissao, lucro_comission):
    """Função que arruma erros para cálculos de participação no lucro"""
    
    # Substitui valores não numéricos por '0' na coluna especificada
    df[banco_comissao] = pd.to_numeric(df[banco_comissao], errors='coerce').fillna(0)

    # Calcula a comissão com base no produto de 'Comissao_Geral' e a coluna de comissão especificada
    df[lucro_comission] = (df['Comissao_Geral'] * df[banco_comissao]).round(2)

# Criando colunas da participação dos lucros:
process_commission_column(planilha_geral, 'Comissao_1', 'Comission_1')
process_commission_column(planilha_geral, 'Comissao_2', 'Comission_2')
process_commission_column(planilha_geral, 'Comissao_3', 'Comission_3')

#save_df_to_excel(planilha_geral, f"{data_arquivo}_planilha_geral_2.xlsx", nome_aba='Sheet1')


print("\nVerique o arquivo 'valores negativos' com os valores que nao irao ao arquivo para pagamento.\n")


Verique o arquivo 'valores negativos' com os valores que nao irao ao arquivo para pagamento.



/var/folders/vr/3ylp0mwj1c37m5vjr6slv2qc0000gn/T/ipykernel_19816/1900742970.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  planilha_geral = pd.merge(consolidado_pgto, banco_de_dados, on="UUID do motorista", how="left").fillna(0)
/var/folders/vr/3ylp0mwj1c37m5vjr6slv2qc0000gn/T/ipykernel_19816/1900742970.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  com_aluguel['key'] = com_aluguel['IBAN'].astype(str) + com_aluguel['DESCONTO ALUGUEL SEMANA'].astype(str)
/var/folders/vr/3ylp0mwj1c37m5vjr6slv2qc0000gn/T/ipykernel_19816/19

<h6>celula_4 - relatorios</h6>

In [5]:
######## ---------------------------- RELATORIO 1 - Começo --------------------------------- ###########
# Selecinando as colunas para o relatorio
relatorio_geral_columns = ['Arq_origem', 'Origem_Ref', 'UUID do motorista', 'first', 'last', 'IBAN', 'Salario Bruto', 'Percentual', 'Valor a Pagar', 'Socio_2', 'Socio_3', 'Valor Comissão']
relatorio_geral = planilha_geral[relatorio_geral_columns]

# Criar a coluna 'NOME NOVO' concatenando 'first' e 'last' com um espaço entre eles
relatorio_geral['NOME NOVO'] = relatorio_geral['first'] + ' ' + relatorio_geral['last']

# Remover as colunas 'first' e 'last'
relatorio_geral = relatorio_geral.drop(columns=['first', 'last'])

# Calculate sums for specific columns
total_row = {
    'UUID do motorista': 'Total',
    'NOME NOVO': '-',
    'Salario Bruto': relatorio_geral['Salario Bruto'].sum(),
    'Valor a Pagar': relatorio_geral['Valor a Pagar'].sum(),
    'Valor Comissão': relatorio_geral['Valor Comissão'].sum()
}

# Append the total row to the DataFrame
relatorio_geral = pd.concat([relatorio_geral, pd.DataFrame([total_row])], ignore_index=True)

save_df_to_excel(relatorio_geral, f"{data_arquivo}_relatorio_geral.xlsx", nome_aba='Sheet1')
######## ---------------------------- RELATORIO 1  -  Fim  --------------------------------- ###########


######## ---------------------------- RELATORIO 1.1 - Top drivers --------------------------------- ###########
from openpyxl.styles import Font

top_5 = relatorio_geral.groupby('Arq_origem', group_keys=False).apply(
    lambda x: x.nlargest(5, 'Salario Bruto')
)

#top_20 = top_20[~top_20['UUID do motorista'].astype(str).str.contains('Total')]


save_df_to_excel(top_5, f"{data_arquivo}_top_5.xlsx", nome_aba='Sheet1')

######## ---------------------------- RELATORIO 1.1 - Top drivers  --------------------------------- ###########


######## ---------------------------- RELATORIO 2 - LOCAL - Começo --------------------------------- ###########
### Relatorio de Analises Comissoes Local
# Ensure 'Valor Comissão' is numeric
planilha_geral['Valor Comissão'] = pd.to_numeric(planilha_geral['Valor Comissão'], errors='coerce')
planilha_geral[['Socio_1', 'Socio_2']] = planilha_geral[['Socio_1', 'Socio_2']].fillna('-')
planilha_geral['Socio_3'] = planilha_geral['Socio_3'].fillna('NOVO PESSOAL')

# Copia valores de uma coluna para a outra
planilha_geral.loc[planilha_geral['Socio_1'] == '-', 'Comission_1'] = planilha_geral['Valor Comissão']

# Criando tabelas dinamicas para consolidar as informacoes de comissao
pivot_comisson1 = planilha_geral.pivot_table(index='Socio_1', columns='Arq_origem', values='Comission_1', aggfunc='sum', fill_value=0).reset_index().rename(columns={'Socio_1': 'Socio'})
pivot_comisson2 = planilha_geral.pivot_table(index='Socio_2', columns='Arq_origem', values='Comission_2', aggfunc='sum', fill_value=0).reset_index().rename(columns={'Socio_2': 'Socio'})
pivot_comisson3 = planilha_geral.pivot_table(index='Socio_3', columns='Arq_origem', values='Comission_3', aggfunc='sum', fill_value=0).reset_index().rename(columns={'Socio_3': 'Socio'})

# Concatenate all three DataFrames vertically
relatorio_comissao_local_grouped = pd.concat([pivot_comisson1, pivot_comisson2, pivot_comisson3], ignore_index=True)

# Update name of the new drivers
relatorio_comissao_local_grouped['Socio'] = relatorio_comissao_local_grouped['Socio'].replace('-', 'NOVO PESSOAL')

# Agrupar por 'Socio' e somar as colunas numéricas para conseguir organizar o arquivo
relatorio_comissao_local = relatorio_comissao_local_grouped.groupby('Socio').sum().reset_index()

# Separando linhas principais para o top
priority_names = ["Guilherme", "NOVO PESSOAL", "Vinicio"]

# Separate rows that match the priority names
priority_rows = relatorio_comissao_local[relatorio_comissao_local['Socio'].isin(priority_names)]

# Separate the remaining rows
remaining_rows = relatorio_comissao_local[~relatorio_comissao_local['Socio'].isin(priority_names)]

# Sort the remaining rows as needed (e.g., alphabetically or by another column)
remaining_rows = remaining_rows.sort_values(by=['Socio'])

# Concatenate priority rows on top of sorted remaining rows
relatorio_comissao_local = pd.concat([priority_rows, remaining_rows], ignore_index=True)

# Add a column 'Total' summing all numeric columns horizontally (excluding 'Socio')
numeric_cols = relatorio_comissao_local.columns.drop('Socio')
relatorio_comissao_local['Total'] = relatorio_comissao_local[numeric_cols].sum(axis=1)

# Ensure all numeric columns are correctly identified
numeric_cols = relatorio_comissao_local.select_dtypes(include='number').columns

# Compute the sum of numeric columns
totals = relatorio_comissao_local[numeric_cols].sum()

# Create a new row as a DataFrame
total_row = pd.DataFrame([['Total Geral'] + totals.tolist()], columns=['Socio'] + list(numeric_cols))

# Append the total row to the original DataFrame
relatorio_comissao_local = pd.concat([relatorio_comissao_local, total_row], ignore_index=True)

# Filter above zero
relatorio_comissao_local = relatorio_comissao_local[relatorio_comissao_local['Total'] > 0]


########################################### Salario total Tabela ###########################################
# Pivot table do salario bruto - analise LOCAL
pivot_comisson4 = planilha_geral.pivot_table(
    columns='Arq_origem', values='Salario Bruto', aggfunc='sum', fill_value=0
).reset_index().rename(columns={'index': 'Socio'})

# Create a new column "Total" summing all numeric columns
pivot_comisson4['Total'] = pivot_comisson4.select_dtypes(include='number').sum(axis=1)

# Concatenate all three DataFrames vertically
relatorio_comissao_local = pd.concat([relatorio_comissao_local, pivot_comisson4], ignore_index=True)

# Select numeric columns
numeric_cols = relatorio_comissao_local.select_dtypes(include='number').columns

# Compute the difference between the last row and the second-to-last row
diferenca_values = relatorio_comissao_local.iloc[-1, :][numeric_cols] - relatorio_comissao_local.iloc[-2, :][numeric_cols]

# Create the "Diferença" row
diferenca_row = pd.DataFrame([['Diferença'] + diferenca_values.tolist()], columns=['Socio'] + list(numeric_cols))

# Append the "Diferença" row to the DataFrame
relatorio_comissao_local = pd.concat([relatorio_comissao_local, diferenca_row], ignore_index=True)

save_df_to_excel(relatorio_comissao_local, f"{data_arquivo}_relatorio_comissao_local.xlsx", nome_aba='Sheet1')
######## ---------------------------- RELATORIO 2  -  Fim  --------------------------------- ###########



######## ---------------------------- RELATORIO 3 - POR SOCIO - Começo --------------------------------- ###########
########################################### Analise Rest ###########################################
# Filter rows where 'Socio_2' is 'Vinicio'
vinicio_socio = planilha_geral[planilha_geral['Socio_2'] == 'Vinicio']

# Create pivot table for 'Vinicio'
pivot_socio_vinicio = vinicio_socio.pivot_table(
    index='Socio_3', 
    values=['Salario Bruto', 'Comission_1', 'Comission_2', 'Comission_3'], 
    aggfunc='sum', 
    fill_value=0
).reset_index()

# Rename columns to Vinicio Analysis
pivot_socio_vinicio = pivot_socio_vinicio.rename(columns={
    'Socio_3': 'Comissao Analises',
    'Comission_1': 'Guilherme',
    'Comission_2': 'Vinicio',
    'Comission_3': 'Socio'
})

# Filter rows where 'Socio_2' is NOT 'Vinicio'
outros_socio2 = planilha_geral[planilha_geral['Socio_2'] != 'Vinicio']

# List to store pivot tables
pivot_list = [pivot_socio_vinicio]  # Start with Vinicio's pivot table

# Ensure there are rows to process
if not outros_socio2.empty:
    # Iterate through unique 'Socio_2' Analysis
    for socio in outros_socio2['Socio_2'].unique():
        # Create a new DataFrame for each 'Socio_2' value
        df_socio = outros_socio2[outros_socio2['Socio_2'] == socio].copy()
        
        # Create pivot table
        pivot_nova2 = df_socio.pivot_table(
            index='Socio_3', 
            values=['Salario Bruto', 'Comission_1', 'Comission_2', 'Comission_3'], 
            aggfunc='sum', 
            fill_value=0
        ).reset_index()

        # Rename columns
        pivot_nova2 = pivot_nova2.rename(columns={
            'Socio_3': 'Comissao Analises',
            'Comission_1': 'Guilherme',
            'Comission_2': f'{socio}',  # Use unique Socio_2 name as column header
            'Comission_3': 'Socio'
        })

        # Append to the list
        pivot_list.append(pivot_nova2)

# Concatenate all pivot tables in the list
relatorio_comissao_socios_temp = pd.concat(pivot_list, ignore_index=True)

# Replace '-' with 'Guilherme' in 'Comissao Analises'
relatorio_comissao_socios_temp['Comissao Analises'] = relatorio_comissao_socios_temp['Comissao Analises'].replace('-', 'Guilherme')

# Grouping by 'Comissao Analises' and sorting alphabetically
relatorio_comissao_socios = relatorio_comissao_socios_temp.groupby('Comissao Analises').sum().reset_index().sort_values(by='Comissao Analises')

# Check if 'NOVO PESSOAL' exists in the 'Comissao Analises' column
if 'NOVO PESSOAL' in relatorio_comissao_socios['Comissao Analises'].values:
    # Move 'NOVO PESSOAL' to the top
    relatorio_comissao_socios = pd.concat([
        relatorio_comissao_socios[relatorio_comissao_socios['Comissao Analises'] == 'NOVO PESSOAL'],
        relatorio_comissao_socios[relatorio_comissao_socios['Comissao Analises'] != 'NOVO PESSOAL']
    ], ignore_index=True)


# Move 'Guilherme' to the top
relatorio_comissao_socios = pd.concat([
    relatorio_comissao_socios[relatorio_comissao_socios['Comissao Analises'] == 'Guilherme'],
    relatorio_comissao_socios[relatorio_comissao_socios['Comissao Analises'] != 'Guilherme']
])

# Check if column '-' exists and drop it
if '-' in relatorio_comissao_socios.columns:
    relatorio_comissao_socios = relatorio_comissao_socios.drop(columns=['-'])

# Move 'Salario Bruto' to index 1
cols = list(relatorio_comissao_socios.columns)
cols.remove('Salario Bruto')  # Remove from the original position
cols.insert(1, 'Salario Bruto')  # Insert at index 1
relatorio_comissao_socios = relatorio_comissao_socios[cols]

# Ensure there are rows to process
if not outros_socio2.empty:
    # Selecionar todas as colunas numéricas, exceto 'Salario Bruto'
    numeric_cols = [col for col in relatorio_comissao_socios.select_dtypes(include='number').columns if col != 'Salario Bruto']

    # Calculate 'Total Comissao' summing all numeric columns except 'Salario Bruto'
    relatorio_comissao_socios["Total Comissao"] = relatorio_comissao_socios[numeric_cols].sum(axis=1)

    # Define fixed column order
    first_cols = ["Comissao Analises", "Salario Bruto", "Total Comissao", "Guilherme", "Vinicio", "Socio"]

    # Identify all other columns dynamically (excluding the first ones)
    other_cols = [col for col in relatorio_comissao_socios.columns if col not in first_cols]

    # Define the final column order
    final_order = first_cols + other_cols

    # Reorder DataFrame
    relatorio_comissao_socios = relatorio_comissao_socios[final_order]


# CASO SO EXISTA GUILHERME E VINICIO NA SEMANA
else:
    # Add 'Total Comissao' at index 2 (summing specified columns)
    relatorio_comissao_socios.insert(2, 'Total Comissao', 
                                     relatorio_comissao_socios[['Guilherme', 'Vinicio', 'Socio']].sum(axis=1))


# Calcular a soma das colunas numéricas
total_row = relatorio_comissao_socios.select_dtypes(include='number').sum().to_frame().T

# Adicionar um rótulo na coluna 'Comissao Analises' para identificar a linha de totais
total_row.insert(0, 'Comissao Analises', 'Total')

# Garantir que as colunas estejam alinhadas antes de concatenar
total_row = total_row.reindex(columns=relatorio_comissao_socios.columns, fill_value='')

# Anexar a linha de totais ao DataFrame
relatorio_comissao_socios = pd.concat([relatorio_comissao_socios, total_row], ignore_index=True)

# Fill NaN values with 0
relatorio_comissao_socios = relatorio_comissao_socios.fillna(0)

save_df_to_excel(relatorio_comissao_socios, f"{data_arquivo}_relatorio_comissao_socios.xlsx", nome_aba='Sheet1')
######## ---------------------------- RELATORIO 3  -  Fim  --------------------------------- ###########


print('\n- Planilha geral de dados e relatorio criados com sucesso.\n')

# Lista de colunas a serem analisadas
analises_columns = ['UUID do motorista', 'Origem_Ref', 'NOME', 'IBAN', 'Arq_origem', 'Salario Bruto', 'Comissao_Geral', 'DESCONTO ALUGUEL SEMANA', 'Deve(+)','DESC. TRANSF BANCARIA', 'Valor a Pagar']

# Seleciona as colunas relevantes do DataFrame original
arquivo_analises = planilha_geral[analises_columns]

# Reinicia o índice do DataFrame para garantir que seja uma sequência numérica
arquivo_analises.reset_index(drop=True, inplace=True)

# Agrupa os dados pela coluna 'Origem_Ref' e calcula a soma das outras colunas
grouped_analises = arquivo_analises.groupby('Origem_Ref').sum(numeric_only=True)

# Reinicia o índice após o agrupamento para converter 'Origem_Ref' de índice para uma coluna
grouped_analises.reset_index(inplace=True)

# Salva o DataFrame em um arquivo Excel
#save_df_to_excel(grouped_analises, f"{data_arquivo}_Resumo por app.xlsx", 'Resumo por app')

# Retorna o DataFrame agrupado
grouped_analises


- Planilha geral de dados e relatorio criados com sucesso.



/var/folders/vr/3ylp0mwj1c37m5vjr6slv2qc0000gn/T/ipykernel_19816/3237209316.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relatorio_geral['NOME NOVO'] = relatorio_geral['first'] + ' ' + relatorio_geral['last']
/var/folders/vr/3ylp0mwj1c37m5vjr6slv2qc0000gn/T/ipykernel_19816/3237209316.py:31: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top_5 = relatorio_geral.groupby('Arq_origem', group_keys=False).apply(


,Origem_Ref,Salario Bruto,Comissao_Geral,DESCONTO ALUGUEL SEMANA,Deve(+),DESC. TRANSF BANCARIA,Valor a Pagar
0,Bolt,30654.21,3015.72,0.0,0.0,771.2,26867.29
1,Uber,210244.55,20701.36,25.0,0.0,1692.8,187825.39


# Arquivo de drivers para pagamento
<h6>celula_5</h6>

In [6]:
########################### PRIMEIRA ANALISE - LUCRO INDIVIDUAL DOS PARCEIROS ###########################
# Separar colunas para arquivo individual dos socios
arquivo_socio_columns = ['UUID do motorista', 'first', 'last', 'Salario Bruto', 'Arq_origem']
arquivo_socio = planilha_geral[arquivo_socio_columns]

# Truncar as colunas 'first' e 'last' para no máximo 16 caracteres
arquivo_socio['first'] = arquivo_socio['first'].str.slice(0, 16)
arquivo_socio['last'] = arquivo_socio['last'].str.slice(0, 16)


# Criando o arquivo para a coluna comission 2 com os valores based da coluna comissao 2
table_1_temp = pd.pivot_table(planilha_geral, values='Comission_1', index=['Socio_1'], 
                           columns=['UUID do motorista'], fill_value=0, aggfunc='sum').reset_index()

# Filter rows where 'Socio_1' is '-'
table_1 = table_1_temp[table_1_temp['Socio_1'] == '-'].copy()  # Use .copy() to avoid SettingWithCopyWarning
table_1 = table_1.rename(columns={'Socio_1': 'Socio'})
table_1['Socio'] = 'New'

# Criando o arquivo para a coluna comission 2 com os valores based da coluna comissao 2
table_2 = pd.pivot_table(planilha_geral, values='Comission_2', index=['Socio_2'], 
                           columns=['UUID do motorista'], fill_value=0, aggfunc='sum').reset_index()

# Renomeando a coluna para o concat funcionar
table_2 = table_2.rename(columns={'Socio_2': 'Socio'})

# Criando o arquivo para a coluna comission 3 com os valores based da coluna comissao 3
table_3 = pd.pivot_table(planilha_geral, values='Comission_3', index=['Socio_3'], 
                           columns=['UUID do motorista'], fill_value=0, aggfunc='sum').reset_index()

# Renomeando a coluna para o concat funcionar
table_3 = table_3.rename(columns={'Socio_3': 'Socio'})

# Juntando os arquivos dos socios em uma tabela
merged_socios = pd.concat([table_1, table_2, table_3], ignore_index=True).fillna(0)

# Agrupando parceiros em um resumo com nomes unicos
merged_socios = merged_socios.groupby('Socio').agg('sum').reset_index()
merged_socios_transposed = merged_socios.transpose().reset_index()

# Arrumando o cabeçalho e reestruturando os arquivos.
merged_socios_transposed.columns = merged_socios_transposed.iloc[0]  # Set the first row as column names
merged_socios_transposed = merged_socios_transposed[1:]  # Remove the first row
merged_socios_transposed = merged_socios_transposed.rename(columns={'Socio': 'UUID do motorista'})


# Função para salvar DataFrame como PDF com várias páginas
def save_to_pdf(df, filename):
    pdf = SimpleDocTemplate(filename, pagesize=A4)
    elements = []
    
    # Convert the DataFrame to a list of lists
    data = [df.columns.tolist()] + df.values.tolist()
    
    # Create a table with the content of the DataFrame
    table = Table(data)
    
    # Calculate column widths to fit within the A4 page width
    column_widths = [pdf.width / len(df.columns) for _ in df.columns]
    table._argW = column_widths
    
    # Set the table style
    table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, -1), 8),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('GRID', (0, 0), (-1, -1), 0.5, colors.black),
    ]))
    
    elements.append(table)
    pdf.build(elements)
    
    
# Lendo cada coluna e procurando somente por colunas com valores acima de 0
for column in merged_socios_transposed.columns[1:]:
    
    # Filtrando linhas onde o valor da coluna é maior que 0
    filtered_df = merged_socios_transposed[merged_socios_transposed[column] > 0]

    # Cria um nome de arquivo para cada coluna
    filename_temp = f'{column}_individual_{data}.pdf'
    filename = os.path.join(new_folder_path, filename_temp)
    
    # Salva 'UUID do motorista' e a coluna atual em PDF, apenas linhas onde coluna > 0
    filtered_df = filtered_df[['UUID do motorista', column]]
    
    # Junta arquivos com as colunas escolhidas da planilha de dados
    arquivo_separado = pd.merge(filtered_df, arquivo_socio, on='UUID do motorista', how='left')
    
    # Define the correct column order as a list
    order_recibo_col = ['Arq_origem', 'first', 'last', 'Salario Bruto', column]  # Fixing the structure

    # Reorder the DataFrame columns
    arquivo_separado = arquivo_separado[order_recibo_col]

    # Calculate total sum for 'Salario Bruto' and column
    total_salario = round(arquivo_separado['Salario Bruto'].sum(), 2)
    total_col = round(arquivo_separado[column].sum(), 2)

    # Create the total row with required values
    total_row = pd.DataFrame({
        'Arq_origem': ['Total'],  # Set 'Total' in 'Arq_origem'
        'first': ['-'],           # Set '-' in 'first'
        'last': ['-'],            # Set '-' in 'last'
        'Salario Bruto': [total_salario],  # Sum for 'Salario Bruto'
        column: [total_col]       # Sum for the specific column
    })

    # Append the total row to the DataFrame
    arquivo_separado = pd.concat([arquivo_separado, total_row], ignore_index=True)

    # Checa se o total eh a cima de 0
    if arquivo_separado[column].sum() > 0 and column != 'New':
        # Salva o arquivo em PDF
        save_to_pdf(arquivo_separado, filename)
        
    #else:
        #agora_vai = pd.DataFrame(arquivo_separado)
        #save_df_to_excel(agora_vai, f"{data_arquivo}_PESSOAL NOVO.xlsx", nome_aba='Sheet1')
       


###########################  ZIPPAR E LIMPAR FOLDER  ###########################
# Define the directory where the PDFs are located
#pdf_directory = os.getcwd()  # Change this if needed
# Find all PDF files in the directory
pdf_files = glob.glob(os.path.join(new_folder_path, "*.pdf"))

# Define the name of the ZIP file
zip_filename_temp = "RECIBOS SOCIOS.zip"
zip_filename = os.path.join(new_folder_path, zip_filename_temp)

# Create a ZIP file and add all PDF files
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for pdf in pdf_files:
        zipf.write(pdf, os.path.basename(pdf))  # Add file to ZIP with its base name

# Delete all original PDF files
for pdf in pdf_files:
    os.remove(pdf)





########################### SEGUNDA ANALISE - ARQUIVO PARA IR AO BANCO ###########################
# Criando a analise das colunas de comissão
pivot_table_1 = planilha_geral.groupby('Socio_1').sum(numeric_only=True).reset_index()

# Calculate 'Comissao_Final' as the sum of individual columns
pivot_table_1['Comissao_Final'] = (pivot_table_1['Comission_1'] + 
                                   pivot_table_1['DESCONTO ALUGUEL SEMANA'] +    
                                   pivot_table_1['DESC. TRANSF BANCARIA'] - 
                                   pivot_table_1['Deve(+)']).round(2)

# Separando a primeira e a ultima coluna com o valor resumo do socio_1 - Guilherme
pivot_table_1 = pivot_table_1.iloc[:, [0, -1]]

# Renomeando colunas
pivot_table_1.rename(columns={pivot_table_1.columns[0]: 'NOME', pivot_table_1.columns[-1]: 'Valor a Pagar'}, inplace=True)

# Separando a coluna do socio_2 e fazer a divisao do lucro
pivot_table_2 = planilha_geral.groupby('Socio_2')['Comission_2'].sum(numeric_only=True).reset_index()
pivot_table_2.rename(columns={pivot_table_2.columns[0]: 'NOME', pivot_table_2.columns[1]: 'Valor a Pagar'}, inplace=True)

# Separando a coluna do socio_3 e fazer a divisao do lucro
pivot_table_3 = planilha_geral.groupby('Socio_3')['Comission_3'].sum(numeric_only=True).reset_index()
pivot_table_3.rename(columns={pivot_table_3.columns[0]: 'NOME', pivot_table_3.columns[1]: 'Valor a Pagar'}, inplace=True)

# Juntando as tres coluns em um so arquivo
merged_pivot = pd.concat([pivot_table_1, pivot_table_2, pivot_table_3], ignore_index=True).fillna(0)

# Agrupando parceiros em um resumo com nomes unicos
merged_pivot = merged_pivot.groupby('NOME').agg({'Valor a Pagar': 'sum' }).reset_index()

# Excluindo zeros da analises
merged_pivot = merged_pivot[merged_pivot['Valor a Pagar'].astype(float) > 0]

# Separando as contas dos socios para criacao do arquivo ao banco
outras_info_subset = outras_info[['NOME', 'IBAN', 'Check.IBAN.Socio']]

# Organizando os dados e renomeando as colunas da panilha - uber
arquivo_comissao_pagamento = pd.merge(merged_pivot, outras_info_subset, on="NOME", how="left").fillna(0)

# Renomeando coluna da validacao do IBAN
arquivo_comissao_pagamento = arquivo_comissao_pagamento.rename(columns={'Check.IBAN.Socio': 'Check.IBAN'})

# Convert 'NOME' column to string and trim spaces
arquivo_comissao_pagamento['NOME'] = arquivo_comissao_pagamento['NOME'].astype(str)

# Adiciona a referencia como Parceiros Comissao
arquivo_comissao_pagamento['Origem_Ref'] = 'Parceiros_Comissao'

# Crie um arquivo retirado da planilha geral com colunas abaixo - valores a serem pagos
pagamento_columns = ['UUID do motorista', 'first', 'last', 'NOME', 'IBAN', 'Arq_origem', 'Check.IBAN', 'Valor a Pagar', 'Valor Comissão', 'Salario Bruto','DESC. TRANSF BANCARIA','Origem_Ref']
arquivo_driver_pagamento = planilha_geral[pagamento_columns]

# Juntando os dois arquivos para pagamento 
arquivo_driver_parceiro = pd.concat([arquivo_comissao_pagamento, arquivo_driver_pagamento], ignore_index=True).fillna(0)

# Verificando quem nao possui IBAN e criando arquivo "sem_iban" se necessario.
sem_iban = arquivo_driver_parceiro[~arquivo_driver_parceiro['Check.IBAN'].astype(str).str.contains('OK')]

####################################  ANALISE SEM IBAN ####################################
# Verificacao com if se sem iban existe
if len(sem_iban) > 0:
    # Remove os Parceiros do relatorio final
    sem_iban = sem_iban[sem_iban['Origem_Ref'] != 'Parceiros_Comissao']

    # Concatenating 'first' and 'last' into 'Novo Nome'
    sem_iban['Novo Nome'] = sem_iban['first'] + ' ' + sem_iban['last']

    # Organizando o relatorio final
    columns_list_sem_iban = [
        "Origem_Ref", "Arq_origem", "NOME", "Novo Nome", "IBAN", 
        "UUID do motorista", "Salario Bruto", "Valor a Pagar", 'Valor Comissão']
    sem_iban = sem_iban[columns_list_sem_iban]


    sem_iban = sem_iban.sort_values(by='Arq_origem').reset_index(drop=True)


    # Define numeric columns to sum
    columns_to_sum = ["Valor a Pagar", "Salario Bruto", 'Valor Comissão']

    # Calculate total sum for the specified columns
    total_values = sem_iban[columns_to_sum].sum().round(2)

    # Create a total row with column names and corresponding sums
    total_row = pd.DataFrame({
        col: ["Total" if i == 0 else total_values.get(col, "-")] if col in columns_to_sum or i == 0 else ["-"]
        for i, col in enumerate(sem_iban.columns)})

    # Append the total row at the bottom of the DataFrame
    sem_iban = pd.concat([sem_iban, total_row], ignore_index=True)

    #################################### Adicionando EXTRA INFORMAÇÕES ####################################
    sem_iban['Socio_1'] = 'Guilherme'
    sem_iban['Socio_2'] = ''
    sem_iban['Socio_3'] = ''

    # Get the total number of rows
    num_rows = len(sem_iban)

    # Generate the Excel formula dynamically for each row em comissao_1
    sem_iban['comissao_1'] = [
        f'=IF(AND(K{i}="", L{i}=""), I{i}, '
        f'IF(AND(K{i}<>"", L{i}=""), I{i}/2, '
        f'IF(AND(L{i}<>"", K{i}=""), I{i}/2, '
        f'IF(AND(K{i}<>"", L{i}<>""), I{i}*0.30, ""))))'
        for i in range(2, num_rows + 2)  # Adjusting row numbers dynamically
    ]

    # Generate formula for comissao_2 correctly
    sem_iban['comissao_2'] = [
        f'=IF(K{i}="", 0, '
        f'IF(AND(K{i}<>"", L{i}=""), I{i}/2, '
        f'IF(AND(K{i}<>"", L{i}<>""), I{i}*0.30, "")))'
        for i in range(2, num_rows + 2)  # Adjusting row numbers dynamically
    ]

    # Generate the Excel formula dynamically for each row in 'comissao_3'
    sem_iban['comissao_3'] = [
        f'=IF(L{i}="", 0, '
        f'IF(AND(L{i}<>"", K{i}=""), I{i}/2, '
        f'IF(AND(K{i}<>"", L{i}<>""), I{i}*0.40, "")))'
        for i in range(2, num_rows + 2)  # Adjusting row numbers dynamically
    ]

    # Set the last row in Socio_1 to "-" - DEVIDO a UNIQUE FORMULA
    sem_iban.loc[num_rows - 1, 'Socio_1'] = ""

    # Define the columns and their corresponding Excel column letters
    columns_to_sum = {'comissao_1': 'M', 'comissao_2': 'N', 'comissao_3': 'O'}

    # Insert the SUM formula dynamically for each column
    for col_name, excel_col in columns_to_sum.items():
        sum_formula = f"=SUM({excel_col}2:{excel_col}{num_rows})"  # Exclude last row from sum
        sem_iban.loc[num_rows - 1, col_name] = sum_formula


    sem_iban_name_file = f"{data_arquivo}_sem_iban.xlsx"

    # Excuta a função previaviamente criada
    save_df_to_excel(sem_iban, sem_iban_name_file, 'sem_iban')
    
    # Create a folder using the current date
    folder_with_date = os.path.join(new_folder_path, sem_iban_name_file)
    re_updating_sem_iban = pd.read_excel(folder_with_date)

    # Save again with specific column widths
    output_file_temp = f"{data_arquivo}_sem_iban.xlsx"
    output_file = os.path.join(new_folder_path, output_file_temp)
    

    with pd.ExcelWriter(output_file, engine="xlsxwriter") as writer:
        
        sem_iban.to_excel(writer, sheet_name='sem_iban', index=False)

        # Get the workbook and worksheet
        workbook = writer.book
        worksheet = writer.sheets['sem_iban']

    
        # Auto-adjust other columns
        for i, col in enumerate(sem_iban.columns):
            if col not in ["comissao_1", "comissao_2", "comissao_3", "NOME"]:
                max_length = max(
                    sem_iban[col].astype(str).map(len).max() if not sem_iban[col].isna().all() else 0,
                    len(col)
                ) + 5  # Add padding
                worksheet.set_column(i, i, max_length)

        #### ANALISE 1 ----------
        # Add the UNIQUE formula in column R, row 1
        worksheet.write_formula("R1", '=FILTER(UNIQUE(J:J), UNIQUE(J:J)<>0)')

        # Add "soma_1" in column S, row 1
        worksheet.write("S1", "soma_1")

        # Insert the IF + SUMIFS formula in column S (rows 2 to 30)
        for i in range(2, 31):  # Rows 2 to 30 (Excel uses 1-based indexing)
            worksheet.write_formula(f"S{i}", 
                f'=IF(OR(R{i}<>0, R{i}<>""), SUMIFS(M:M, J:J, R{i}), "")')


        #### ANALISE 2 ------------
        # Add the UNIQUE formula in column U, row 1
        worksheet.write_formula("U1", "=FILTER(UNIQUE(K:K), UNIQUE(K:K)<>0)")

        # Add "soma_2" in column V, row 1
        worksheet.write("V1", "soma_2")

        # Insert the IF + SUMIFS formula in column U (rows 2 to 30)
        for i in range(2, 31):  # Rows 2 to 30
            worksheet.write_formula(f"V{i}", 
                f'=IF(OR(U{i}<>0, U{i}<>""), SUMIFS(N:N, K:K, U{i}), "")')


        #### ANALISE 3 -----------
        # Add the UNIQUE formula in column X, row 1
        worksheet.write_formula("X1", "=FILTER(UNIQUE(L:L), UNIQUE(L:L)<>0)")

        # Add "soma_2" in column Y, row 1
        worksheet.write("Y1", "soma_3")

        # Insert the IF + SUMIFS formula in column Y (rows 2 to 30)
        for i in range(2, 31):  # Rows 2 to 30
            worksheet.write_formula(f"Y{i}", 
                f'=IF(OR(X{i}<>0, X{i}<>""), SUMIFS(O:O, L:L, X{i}), "")')

        # Define number format for columns M to Y
        number_format = workbook.add_format({'num_format': '#,##0.00'})

        # Apply the number format to columns M to Y
        worksheet.set_column('M:Y', None, number_format)  # None keeps the default width

        # Define column widths manually
        worksheet.set_column("C:C", 20)  # Adjust if 'NOME' is in a different column
        worksheet.set_column("M:M", 13)  # Adjust this if 'comissao_1' is in a different column
        worksheet.set_column("N:N", 13)  # Adjust if 'comissao_2' is in a different column
        worksheet.set_column("O:O", 13)  # Adjust if 'comissao_3' is in a different column
       
        # Save the updated Excel file
        writer.close()
        
    print('\n\n Verifique arquivo "sem_iban" como pessoas que nao entrarão no arquivo para pagamento de pagamento. \n')

########################### QUEBRANDO O ARQUIVO EM DIFERENTE ARQUIVOS PARA RODAR MAIS DE UMA AUTOMATIZACAO
# Criando arquivo que ira para o banco
vai_para_banco = arquivo_driver_parceiro[arquivo_driver_parceiro['Check.IBAN'].astype(str).str.contains('OK')]

/var/folders/vr/3ylp0mwj1c37m5vjr6slv2qc0000gn/T/ipykernel_19816/1382232230.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  arquivo_socio['first'] = arquivo_socio['first'].str.slice(0, 16)
/var/folders/vr/3ylp0mwj1c37m5vjr6slv2qc0000gn/T/ipykernel_19816/1382232230.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  arquivo_socio['last'] = arquivo_socio['last'].str.slice(0, 16)




 Verifique arquivo "sem_iban" como pessoas que nao entrarão no arquivo para pagamento de pagamento. 



/opt/anaconda3/lib/python3.12/site-packages/xlsxwriter/workbook.py:369: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


<h6>celula_6</h6>

In [7]:
# Total number of rows in the DataFrame
total_rows = len(vai_para_banco)

# Define how many parts you want to split your DataFrame into
number_of_splits = 3

# Calculate the number of rows for each file, and the remainder to add to the first few files
base_size = total_rows // number_of_splits
remainder = total_rows % number_of_splits

# Create an array to hold the size of each split
split_sizes = [base_size + 1 if i < remainder else base_size for i in range(number_of_splits)]

# Initialize the starting index
start_index = 0

# Initialize an empty list to hold the DataFrame segments
segments = []

# Split the DataFrame and process each part
for i, size in enumerate(split_sizes):
    # Define the end index for this segment
    end_index = start_index + size

    # Extract the segment of the DataFrame and make a copy to avoid SettingWithCopyWarning
    segment = vai_para_banco.iloc[start_index:end_index].copy()

    # Create a column with the file name
    segment['File_Name'] = f"Banco{i + 1}"

    # Add the segment to the list
    segments.append(segment)

    # Update the start index for the next segment
    start_index = end_index

# Concatenate all segments into a single DataFrame
banco_consolidado = pd.concat(segments, ignore_index=True)

# Select only the visible columns
visible_columns = ["NOME", "Valor a Pagar", "IBAN", "File_Name"]
banco_consolidado_visible = banco_consolidado[visible_columns]

# Define the file path for saving the Excel file
banco_consolidado_visible_temp = f"{data_arquivo}_Arquivo_Banco_consolidado.xlsx"
banco_consolidado_visible_path = os.path.join(new_folder_path, banco_consolidado_visible_temp)

# Save the consolidated DataFrame to an Excel file
banco_consolidado_visible.to_excel(banco_consolidado_visible_path, index=False)
    
print('\n\n\n *** Arquivo para automatizacao pronto ***')
print(' *** Depois de tudo confirmado, avançe para rodar o XML *** \n\n\n')






 *** Arquivo para automatizacao pronto ***
 *** Depois de tudo confirmado, avançe para rodar o XML *** 





In [8]:
import re
import pandas as pd
import os

# Define the file path for saving the Excel file
banco_consolidado_visible_temp = f"{data_arquivo}_Arquivo_Banco_consolidado.xlsx"
banco_consolidado_visible_path = os.path.join(new_folder_path, banco_consolidado_visible_temp)

# Load data from Excel file
payments_temp = pd.read_excel(banco_consolidado_visible_path)

# Select the relevant columns
col_seguimentos_keep = ["NOME", "Valor a Pagar", "IBAN", "File_Name"]
payments_region = payments_temp[col_seguimentos_keep]

# Loop through each unique region (based on 'File_Name')
for file_name_col in payments_region['File_Name'].unique():
    # Filter the DataFrame for the current region and make a copy
    payments = payments_region[payments_region['File_Name'] == file_name_col].copy()

    # Rename columns for consistency
    payments.columns = ["Creditor", "Amount", "IBAN", "Payment_ID"]

    # Convert all values in the 'Creditor' column to uppercase and limit the length to 28 characters
    payments["Creditor"] = payments["Creditor"].str.upper().str.slice(0, 28)

    # Convert IBAN values to uppercase
    payments["IBAN"] = payments["IBAN"].str.upper()

    # Ensure 'Amount' is of type float
    payments["Amount"] = payments["Amount"].astype(float)

    # Assign the same 'Payment_ID' for all rows
    payments["Payment_ID"] = "PAGAMENTO SEMANAL"

    # XML structure
    root = ET.Element("Document", {
        "xmlns": "urn:iso:std:iso:20022:tech:xsd:pain.001.001.03",
        "xmlns:xsi": "http://www.w3.org/2001/XMLSchema-instance"
    })
    cstmr_cdt_trf_initn = ET.SubElement(root, "CstmrCdtTrfInitn")

    # Group Header
    grp_hdr = ET.SubElement(cstmr_cdt_trf_initn, "GrpHdr")

    # Generate timestamp in format YYYYMMDDHHMM
    timestamp = datetime.now().strftime('%Y%m%d%H%M')
    # Dynamically assign it to your XML element
    ET.SubElement(grp_hdr, "MsgId").text = f"UPROFIT911B-{timestamp}" # ATUALIZADO
    #ET.SubElement(grp_hdr, "MsgId").text = "UPROFIT911B-2025012111123062"

    ET.SubElement(grp_hdr, "CreDtTm").text = datetime.now().strftime("%Y-%m-%dT%H:%M:%S")

    ET.SubElement(grp_hdr, "NbOfTxs").text = str(len(payments))
    ET.SubElement(grp_hdr, "CtrlSum").text = f"{payments['Amount'].sum():.2f}"

    initg_pty = ET.SubElement(grp_hdr, "InitgPty")
    ET.SubElement(initg_pty, "Nm").text = "3WAQUISECONDUZ UNIPESSOAL LDA"

    # Adding ID structure
    initg_pty_id = ET.SubElement(initg_pty, "Id")
    prvt_id = ET.SubElement(initg_pty_id, "PrvtId")
    othr = ET.SubElement(prvt_id, "Othr")
    ET.SubElement(othr, "Id").text = "516574361"

    # Payment Information
    pmt_inf = ET.SubElement(cstmr_cdt_trf_initn, "PmtInf")


    ET.SubElement(pmt_inf, "PmtInfId").text = f"UPROFIT911B-{timestamp}"
    #ET.SubElement(pmt_inf, "PmtInfId").text = "UPROFIT911B-20250121111230"

    ET.SubElement(pmt_inf, "PmtMtd").text = "TRF"
    ET.SubElement(pmt_inf, "NbOfTxs").text = str(len(payments))
    ET.SubElement(pmt_inf, "CtrlSum").text = f"{payments['Amount'].sum():.2f}"

    pmt_tp_inf = ET.SubElement(pmt_inf, "PmtTpInf")

    # IMEDIATA TAG
    lcl_instrm = ET.SubElement(pmt_tp_inf, "LclInstrm")
    ET.SubElement(lcl_instrm, "Prtry").text = "INST"

    ctgy_purp = ET.SubElement(pmt_tp_inf, "CtgyPurp")
    ET.SubElement(ctgy_purp, "Cd").text = "SALA"


    # Current date formatted as YYYY-MM-DD
    current_date = datetime.today().strftime('%Y-%m-%d')

    # Dynamically assign it to your XML element
    ET.SubElement(pmt_inf, "ReqdExctnDt").text = current_date

    # Debtor Information
    dbtr = ET.SubElement(pmt_inf, "Dbtr")
    ET.SubElement(dbtr, "Nm").text = "3WAQUISECONDUZ UNIPESSOAL LDA"

    dbtr_acct = ET.SubElement(pmt_inf, "DbtrAcct")
    dbtr_acct_id = ET.SubElement(dbtr_acct, "Id")
    ET.SubElement(dbtr_acct_id, "IBAN").text = "PT50003300004565690070705"

    # Adding Debtor Agent information
    dbtr_agt = ET.SubElement(pmt_inf, "DbtrAgt")
    ET.SubElement(dbtr_agt, "FinInstnId").text = ""

    # Loop through payments DataFrame
    for row in payments.itertuples(index=False):
        cdt_trf_tx_inf = ET.SubElement(pmt_inf, "CdtTrfTxInf")

        pmt_id = ET.SubElement(cdt_trf_tx_inf, "PmtId")
        ET.SubElement(pmt_id, "EndToEndId").text = row.Payment_ID if pd.notna(row.Payment_ID) else "PAGAMENTO"

        amt = ET.SubElement(cdt_trf_tx_inf, "Amt")
        ET.SubElement(amt, "InstdAmt", Ccy="EUR").text = f"{row.Amount:.2f}"

        cdtr_agt = ET.SubElement(cdt_trf_tx_inf, "CdtrAgt")
        fin_instn_id = ET.SubElement(cdtr_agt, "FinInstnId")
        fin_instn_id.text = "" 

        cdtr = ET.SubElement(cdt_trf_tx_inf, "Cdtr")
        ET.SubElement(cdtr, "Nm").text = row.Creditor

        cdtr_acct = ET.SubElement(cdt_trf_tx_inf, "CdtrAcct")
        cdtr_acct_id = ET.SubElement(cdtr_acct, "Id")
        ET.SubElement(cdtr_acct_id, "IBAN").text = row.IBAN

    ## Converter data para yyyy-mm-dd
    new_data = datetime.strptime(data_arquivo, "%Y%m%d").strftime("%Y-%m-%d")

    # Converte o XML em string
    xml_str = ET.tostring(root, encoding='utf-8')
    parsed = minidom.parseString(xml_str)
    pretty_xml = parsed.toprettyxml(indent="    ")  # indentação de 4 espaços

    # Corrige a primeira linha (declaração XML)
    pretty_xml = pretty_xml.replace('<?xml version="1.0" ?>', '<?xml version="1.0" encoding="utf-8"?>')

    # Divide o XML formatado em linhas
    lines = pretty_xml.splitlines()

    # Junta os atributos da tag <Document> em uma única linha
    for i, line in enumerate(lines):
        if '<Document' in line:
            while not lines[i].strip().endswith('>'):
                lines[i] = lines[i].strip() + ' ' + lines[i + 1].strip()
                del lines[i + 1]
            break

    # Mantém a primeira linha (declaração XML) sem indentação
    final_lines = [lines[0]]  # Primeira linha intacta

    # Aplica 12 espaços nas demais linhas (exceto linhas vazias)
    for line in lines[1:]:
        final_lines.append(' ' * 12 + line if line.strip() else '')

    # Recompõe o XML final
    final_xml = '\n'.join(final_lines)

    # Nome do arquivo XML
    nome_arquivo = f"XML{file_name_col}.xml"
    caminho_arquivo = os.path.join(new_folder_path, nome_arquivo)

    # Salva no arquivo
    with open(caminho_arquivo, 'w', encoding='utf-8') as f:
        f.write(final_xml)

    print(f"XML file generated: {nome_arquivo}")



XML file generated: XMLBanco1.xml
XML file generated: XMLBanco2.xml
XML file generated: XMLBanco3.xml


In [11]:
import re
import pandas as pd
import os

# Nome do arquivo XML
sem_iban_xml_temp = f"{data_arquivo}_sem_iban.xlsx"
sem_iban_xml = os.path.join(new_folder_path, sem_iban_xml_temp)

# Load data
payments_sem_iban = pd.read_excel(sem_iban_xml)

# Keep specific columns
col_seguimentos_keep = ["Novo Nome", "Valor a Pagar", "IBAN", "Arq_origem"]
payments_sem_iban = payments_sem_iban[col_seguimentos_keep]

# Rename columns for consistency
payments_sem_iban.columns = ["Creditor", "Amount", "IBAN", "Payment_ID"]

# Convert 'Creditor' to uppercase and slice
payments_sem_iban["Creditor"] = payments_sem_iban["Creditor"].str.upper().str.slice(0, 28)

# Clean 'IBAN' column (keep only letters and numbers, remove special characters)
payments_sem_iban["IBAN"] = payments_sem_iban["IBAN"].fillna('0')
payments_sem_iban["IBAN"] = payments_sem_iban["IBAN"].astype(str).str.replace(r'[^A-Za-z0-9]', '', regex=True)
payments_sem_iban["IBAN"] = payments_sem_iban["IBAN"].str.upper()



# Define a function to check IBAN rules
def iban_check(iban):
    if pd.isna(iban) or iban == '':
        return False
    first_char = iban[0]
    if first_char.isalpha():
        if first_char == 'P':
            return len(iban) == 25
        else:
            return True
    else:
        return False

# Apply the function to filter correct IBANs
payments_sem_iban_validos = payments_sem_iban[payments_sem_iban['IBAN'].apply(iban_check)]

# Apply the function to filter invalid IBANs
payments_sem_iban_nao_enviados = payments_sem_iban[~payments_sem_iban['IBAN'].apply(iban_check)]


# Save invalid IBANs
save_df_to_excel(payments_sem_iban_nao_enviados, f"payments_sem_iban_nao_enviados.xlsx", 'ao banco')

# Continue with valid payments
payments_sem_iban_validos["Amount"] = payments_sem_iban_validos["Amount"].astype(float)
payments_sem_iban_validos["Payment_ID"] = "PAGAMENTO SEMANAL"

# Save valid payments
save_df_to_excel(payments_sem_iban_validos, f"payments_sem_iban.xlsx", 'ao banco')


payments_sem_iban = payments_sem_iban_validos

# XML structure
root = ET.Element("Document", {
    "xmlns": "urn:iso:std:iso:20022:tech:xsd:pain.001.001.03",
    "xmlns:xsi": "http://www.w3.org/2001/XMLSchema-instance"
})
cstmr_cdt_trf_initn = ET.SubElement(root, "CstmrCdtTrfInitn")

# Group Header
grp_hdr = ET.SubElement(cstmr_cdt_trf_initn, "GrpHdr")

# Generate timestamp in format YYYYMMDDHHMM
timestamp = datetime.now().strftime('%Y%m%d%H%M')
# Dynamically assign it to your XML element
ET.SubElement(grp_hdr, "MsgId").text = f"UPROFIT911B-{timestamp}" # ATUALIZADO
#ET.SubElement(grp_hdr, "MsgId").text = "UPROFIT911B-2025012111123062"

ET.SubElement(grp_hdr, "CreDtTm").text = datetime.now().strftime("%Y-%m-%dT%H:%M:%S")

ET.SubElement(grp_hdr, "NbOfTxs").text = str(len(payments_sem_iban))
ET.SubElement(grp_hdr, "CtrlSum").text = f"{payments_sem_iban['Amount'].sum():.2f}"

initg_pty = ET.SubElement(grp_hdr, "InitgPty")
ET.SubElement(initg_pty, "Nm").text = "3WAQUISECONDUZ UNIPESSOAL LDA"

# Adding ID structure
initg_pty_id = ET.SubElement(initg_pty, "Id")
prvt_id = ET.SubElement(initg_pty_id, "PrvtId")
othr = ET.SubElement(prvt_id, "Othr")
ET.SubElement(othr, "Id").text = "516574361"

# Payment Information
pmt_inf = ET.SubElement(cstmr_cdt_trf_initn, "PmtInf")


ET.SubElement(pmt_inf, "PmtInfId").text = f"UPROFIT911B-{timestamp}"
#ET.SubElement(pmt_inf, "PmtInfId").text = "UPROFIT911B-20250121111230"

ET.SubElement(pmt_inf, "PmtMtd").text = "TRF"
ET.SubElement(pmt_inf, "NbOfTxs").text = str(len(payments_sem_iban))
ET.SubElement(pmt_inf, "CtrlSum").text = f"{payments_sem_iban['Amount'].sum():.2f}"

pmt_tp_inf = ET.SubElement(pmt_inf, "PmtTpInf")

# IMEDIATA TAG
lcl_instrm = ET.SubElement(pmt_tp_inf, "LclInstrm")
ET.SubElement(lcl_instrm, "Prtry").text = "INST"

ctgy_purp = ET.SubElement(pmt_tp_inf, "CtgyPurp")
ET.SubElement(ctgy_purp, "Cd").text = "SALA"


# Current date formatted as YYYY-MM-DD
current_date = datetime.today().strftime('%Y-%m-%d')

# Dynamically assign it to your XML element
ET.SubElement(pmt_inf, "ReqdExctnDt").text = current_date

# Debtor Information
dbtr = ET.SubElement(pmt_inf, "Dbtr")
ET.SubElement(dbtr, "Nm").text = "3WAQUISECONDUZ UNIPESSOAL LDA"

dbtr_acct = ET.SubElement(pmt_inf, "DbtrAcct")
dbtr_acct_id = ET.SubElement(dbtr_acct, "Id")
ET.SubElement(dbtr_acct_id, "IBAN").text = "PT50003300004565690070705"

# Adding Debtor Agent information
dbtr_agt = ET.SubElement(pmt_inf, "DbtrAgt")
ET.SubElement(dbtr_agt, "FinInstnId").text = ""

# Loop through payments_sem_iban DataFrame
for row in payments_sem_iban.itertuples(index=False):
    cdt_trf_tx_inf = ET.SubElement(pmt_inf, "CdtTrfTxInf")

    pmt_id = ET.SubElement(cdt_trf_tx_inf, "PmtId")
    ET.SubElement(pmt_id, "EndToEndId").text = row.Payment_ID if pd.notna(row.Payment_ID) else "PAGAMENTO"

    amt = ET.SubElement(cdt_trf_tx_inf, "Amt")
    ET.SubElement(amt, "InstdAmt", Ccy="EUR").text = f"{row.Amount:.2f}"

    cdtr_agt = ET.SubElement(cdt_trf_tx_inf, "CdtrAgt")
    fin_instn_id = ET.SubElement(cdtr_agt, "FinInstnId")
    fin_instn_id.text = "" 

    cdtr = ET.SubElement(cdt_trf_tx_inf, "Cdtr")
    ET.SubElement(cdtr, "Nm").text = row.Creditor

    cdtr_acct = ET.SubElement(cdt_trf_tx_inf, "CdtrAcct")
    cdtr_acct_id = ET.SubElement(cdtr_acct, "Id")
    ET.SubElement(cdtr_acct_id, "IBAN").text = row.IBAN

## Converter data para yyyy-mm-dd
new_data = datetime.strptime(data_arquivo, "%Y%m%d").strftime("%Y-%m-%d")

# Converte o XML em string
xml_str = ET.tostring(root, encoding='utf-8')
parsed = minidom.parseString(xml_str)
pretty_xml = parsed.toprettyxml(indent="    ")  # indentação de 4 espaços

# Corrige a primeira linha (declaração XML)
pretty_xml = pretty_xml.replace('<?xml version="1.0" ?>', '<?xml version="1.0" encoding="utf-8"?>')

# Divide o XML formatado em linhas
lines = pretty_xml.splitlines()

# Junta os atributos da tag <Document> em uma única linha
for i, line in enumerate(lines):
    if '<Document' in line:
        while not lines[i].strip().endswith('>'):
            lines[i] = lines[i].strip() + ' ' + lines[i + 1].strip()
            del lines[i + 1]
        break

# Mantém a primeira linha (declaração XML) sem indentação
final_lines = [lines[0]]  # Primeira linha intacta

# Aplica 12 espaços nas demais linhas (exceto linhas vazias)
for line in lines[1:]:
    final_lines.append(' ' * 12 + line if line.strip() else '')

# Recompõe o XML final
final_xml = '\n'.join(final_lines)

# Nome do arquivo XML
sem_iban_batch_temp = f"BATCHSEMIBAN.xml"
sem_iban_batch = os.path.join(new_folder_path, sem_iban_batch_temp)

# Salva no arquivo
with open(sem_iban_batch, 'w', encoding='utf-8') as f:
    f.write(final_xml)

print(f"XML file generated: {sem_iban_batch_temp}")


XML file generated: BATCHSEMIBAN.xml


/var/folders/vr/3ylp0mwj1c37m5vjr6slv2qc0000gn/T/ipykernel_19816/3913374842.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  payments_sem_iban_validos["Amount"] = payments_sem_iban_validos["Amount"].astype(float)
/var/folders/vr/3ylp0mwj1c37m5vjr6slv2qc0000gn/T/ipykernel_19816/3913374842.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  payments_sem_iban_validos["Payment_ID"] = "PAGAMENTO SEMANAL"


In [10]:
import re
import pandas as pd
import os

# Nome do arquivo XML
promo_xml_temp = "PLANILHA PROMOCOES_MODELO-FINAL.xlsx"
promo_xml = os.path.join(new_folder_path, promo_xml_temp)

# Load data
payments_promo = pd.read_excel(promo_xml, sheet_name='XML')

# Keep specific columns
col_seguimentos_keep = ["Novo Nome", "Valor a Pagar", "IBAN", "Arq_origem"]
payments_promo = payments_promo[col_seguimentos_keep]

# Rename columns for consistency
payments_promo.columns = ["Creditor", "Amount", "IBAN", "Payment_ID"]

# Convert 'Creditor' to uppercase and slice
payments_promo["Creditor"] = payments_promo["Creditor"].str.upper().str.slice(0, 28)

# Clean 'IBAN' column (keep only letters and numbers, remove special characters)
payments_promo["IBAN"] = payments_promo["IBAN"].fillna('0')
payments_promo["IBAN"] = payments_promo["IBAN"].astype(str).str.replace(r'[^A-Za-z0-9]', '', regex=True)
payments_promo["IBAN"] = payments_promo["IBAN"].str.upper()

# Define a function to check IBAN rules
def iban_check(iban):
    if pd.isna(iban) or iban == '':
        return False
    first_char = iban[0]
    if first_char.isalpha():
        if first_char == 'P':
            return len(iban) == 25
        else:
            return True
    else:
        return False

# Apply the function to filter correct IBANs
payments_promo_validos = payments_promo[payments_promo['IBAN'].apply(iban_check)]

# Apply the function to filter invalid IBANs
payments_promo_nao_enviados = payments_promo[~payments_promo['IBAN'].apply(iban_check)]

# Save invalid IBANs
save_df_to_excel(payments_promo_nao_enviados, f"payments_promo_nao_enviados.xlsx", 'ao banco')

# Continue with valid payments
payments_promo_validos["Amount"] = payments_promo_validos["Amount"].astype(float)
payments_promo_validos["Payment_ID"] = "PAGAMENTO SEMANAL"

# Save valid payments
save_df_to_excel(payments_promo_validos, f"payments_promo.xlsx", 'ao banco')

payments_promo = payments_promo_validos


# XML structure
root = ET.Element("Document", {
    "xmlns": "urn:iso:std:iso:20022:tech:xsd:pain.001.001.03",
    "xmlns:xsi": "http://www.w3.org/2001/XMLSchema-instance"
})
cstmr_cdt_trf_initn = ET.SubElement(root, "CstmrCdtTrfInitn")

# Group Header
grp_hdr = ET.SubElement(cstmr_cdt_trf_initn, "GrpHdr")

# Generate timestamp in format YYYYMMDDHHMM
timestamp = datetime.now().strftime('%Y%m%d%H%M')
# Dynamically assign it to your XML element
ET.SubElement(grp_hdr, "MsgId").text = f"UPROFIT911B-{timestamp}" # ATUALIZADO
#ET.SubElement(grp_hdr, "MsgId").text = "UPROFIT911B-2025012111123062"

ET.SubElement(grp_hdr, "CreDtTm").text = datetime.now().strftime("%Y-%m-%dT%H:%M:%S")

ET.SubElement(grp_hdr, "NbOfTxs").text = str(len(payments_promo))
ET.SubElement(grp_hdr, "CtrlSum").text = f"{payments_promo['Amount'].sum():.2f}"

initg_pty = ET.SubElement(grp_hdr, "InitgPty")
ET.SubElement(initg_pty, "Nm").text = "3WAQUISECONDUZ UNIPESSOAL LDA"

# Adding ID structure
initg_pty_id = ET.SubElement(initg_pty, "Id")
prvt_id = ET.SubElement(initg_pty_id, "PrvtId")
othr = ET.SubElement(prvt_id, "Othr")
ET.SubElement(othr, "Id").text = "516574361"

# Payment Information
pmt_inf = ET.SubElement(cstmr_cdt_trf_initn, "PmtInf")


ET.SubElement(pmt_inf, "PmtInfId").text = f"UPROFIT911B-{timestamp}"
#ET.SubElement(pmt_inf, "PmtInfId").text = "UPROFIT911B-20250121111230"

ET.SubElement(pmt_inf, "PmtMtd").text = "TRF"
ET.SubElement(pmt_inf, "NbOfTxs").text = str(len(payments_promo))
ET.SubElement(pmt_inf, "CtrlSum").text = f"{payments_promo['Amount'].sum():.2f}"

pmt_tp_inf = ET.SubElement(pmt_inf, "PmtTpInf")

# IMEDIATA TAG
lcl_instrm = ET.SubElement(pmt_tp_inf, "LclInstrm")
ET.SubElement(lcl_instrm, "Prtry").text = "INST"

ctgy_purp = ET.SubElement(pmt_tp_inf, "CtgyPurp")
ET.SubElement(ctgy_purp, "Cd").text = "SALA"


# Current date formatted as YYYY-MM-DD
current_date = datetime.today().strftime('%Y-%m-%d')

# Dynamically assign it to your XML element
ET.SubElement(pmt_inf, "ReqdExctnDt").text = current_date

# Debtor Information
dbtr = ET.SubElement(pmt_inf, "Dbtr")
ET.SubElement(dbtr, "Nm").text = "3WAQUISECONDUZ UNIPESSOAL LDA"

dbtr_acct = ET.SubElement(pmt_inf, "DbtrAcct")
dbtr_acct_id = ET.SubElement(dbtr_acct, "Id")
ET.SubElement(dbtr_acct_id, "IBAN").text = "PT50003300004565690070705"

# Adding Debtor Agent information
dbtr_agt = ET.SubElement(pmt_inf, "DbtrAgt")
ET.SubElement(dbtr_agt, "FinInstnId").text = ""

# Loop through payments_sem_iban DataFrame
for row in payments_promo.itertuples(index=False):
    cdt_trf_tx_inf = ET.SubElement(pmt_inf, "CdtTrfTxInf")

    pmt_id = ET.SubElement(cdt_trf_tx_inf, "PmtId")
    ET.SubElement(pmt_id, "EndToEndId").text = row.Payment_ID if pd.notna(row.Payment_ID) else "PAGAMENTO"

    amt = ET.SubElement(cdt_trf_tx_inf, "Amt")
    ET.SubElement(amt, "InstdAmt", Ccy="EUR").text = f"{row.Amount:.2f}"

    cdtr_agt = ET.SubElement(cdt_trf_tx_inf, "CdtrAgt")
    fin_instn_id = ET.SubElement(cdtr_agt, "FinInstnId")
    fin_instn_id.text = "" 

    cdtr = ET.SubElement(cdt_trf_tx_inf, "Cdtr")
    ET.SubElement(cdtr, "Nm").text = row.Creditor

    cdtr_acct = ET.SubElement(cdt_trf_tx_inf, "CdtrAcct")
    cdtr_acct_id = ET.SubElement(cdtr_acct, "Id")
    ET.SubElement(cdtr_acct_id, "IBAN").text = row.IBAN

## Converter data para yyyy-mm-dd
new_data = datetime.strptime(data_arquivo, "%Y%m%d").strftime("%Y-%m-%d")

# Converte o XML em string
xml_str = ET.tostring(root, encoding='utf-8')
parsed = minidom.parseString(xml_str)
pretty_xml = parsed.toprettyxml(indent="    ")  # indentação de 4 espaços

# Corrige a primeira linha (declaração XML)
pretty_xml = pretty_xml.replace('<?xml version="1.0" ?>', '<?xml version="1.0" encoding="utf-8"?>')

# Divide o XML formatado em linhas
lines = pretty_xml.splitlines()

# Junta os atributos da tag <Document> em uma única linha
for i, line in enumerate(lines):
    if '<Document' in line:
        while not lines[i].strip().endswith('>'):
            lines[i] = lines[i].strip() + ' ' + lines[i + 1].strip()
            del lines[i + 1]
        break

# Mantém a primeira linha (declaração XML) sem indentação
final_lines = [lines[0]]  # Primeira linha intacta

# Aplica 12 espaços nas demais linhas (exceto linhas vazias)
for line in lines[1:]:
    final_lines.append(' ' * 12 + line if line.strip() else '')

# Recompõe o XML final
final_xml = '\n'.join(final_lines)

# Nome do arquivo XML
promo_batch_temp = f"BATCHPROMOCAO.xml"
promo_batch = os.path.join(new_folder_path, promo_batch_temp)

# Salva no arquivo
with open(promo_batch, 'w', encoding='utf-8') as f:
    f.write(final_xml)

print(f"XML file generated: {promo_batch_temp}")


XML file generated: BATCHPROMOCAO.xml
